In [2]:
# 1. Spark 세션 재생성
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("accident-severity-prediction") \
    .config("spark.driver.memory", "8g") \
    .getOrCreate()

In [3]:
# 2. accident_df 다시 로드
import os
cwd = os.getcwd()
a_data_path = os.path.join(cwd, 'learning_spark_data', 'accident' , 'Accident_Information.csv')
a_file_path = f"file:///{a_data_path.replace(os.sep, '/')}"
accident_df = spark.read.csv(a_file_path, inferSchema=True, header=True)

In [12]:
# 불러오기
accident_df = spark.read.csv(a_file_path, inferSchema=True, header=True)

# StringIndexer로 fit 및 transform
from pyspark.ml.feature import StringIndexer

severity_indexer = StringIndexer(inputCol="Accident_Severity", outputCol="label")
severity_indexer_model = severity_indexer.fit(accident_df)

print(severity_indexer_model.labels)  # 레이블 순서 확인용

accident_df = severity_indexer_model.transform(accident_df)
accident_df.select("Accident_Severity", "label").show(10)


['Slight', 'Serious', 'Fatal']
+-----------------+-----+
|Accident_Severity|label|
+-----------------+-----+
|          Serious|  1.0|
|           Slight|  0.0|
|           Slight|  0.0|
|           Slight|  0.0|
|           Slight|  0.0|
|           Slight|  0.0|
|           Slight|  0.0|
|           Slight|  0.0|
|           Slight|  0.0|
|           Slight|  0.0|
+-----------------+-----+
only showing top 10 rows



In [14]:
# vehicle 불러오기
v_data_path = os.path.join(cwd, 'learning_spark_data', 'accident' , 'Vehicle_Information.csv')
v_file_path = f"file:///{v_data_path.replace(os.sep, '/')}"

# 2. CSV 파일 읽기
vehicle_df = spark.read.csv(v_file_path, inferSchema=True, header=True)

# 3. 스키마 확인 (선택)
vehicle_df.printSchema()

root
 |-- Accident_Index: string (nullable = true)
 |-- Age_Band_of_Driver: string (nullable = true)
 |-- Age_of_Vehicle: string (nullable = true)
 |-- Driver_Home_Area_Type: string (nullable = true)
 |-- Driver_IMD_Decile: string (nullable = true)
 |-- Engine_Capacity_.CC.: string (nullable = true)
 |-- Hit_Object_in_Carriageway: string (nullable = true)
 |-- Hit_Object_off_Carriageway: string (nullable = true)
 |-- Journey_Purpose_of_Driver: string (nullable = true)
 |-- Junction_Location: string (nullable = true)
 |-- make: string (nullable = true)
 |-- model: string (nullable = true)
 |-- Propulsion_Code: string (nullable = true)
 |-- Sex_of_Driver: string (nullable = true)
 |-- Skidding_and_Overturning: string (nullable = true)
 |-- Towing_and_Articulation: string (nullable = true)
 |-- Vehicle_Leaving_Carriageway: string (nullable = true)
 |-- Vehicle_Location.Restricted_Lane: string (nullable = true)
 |-- Vehicle_Manoeuvre: string (nullable = true)
 |-- Vehicle_Reference: intege

In [19]:
# vehicle_df rename 
vehicle_df = vehicle_df.withColumnRenamed("Engine_Capacity_.CC.", "Engine_Capacity")

# accident_df와 vehicle_df 조인 하기
df = accident_df.join(vehicle_df, on="Accident_Index", how="inner")

# 선택 컬럼만 뽑기
selected_cols = [
    "label",
    "Age_Band_of_Driver",
    "Sex_of_Driver",
    "Vehicle_Type",
    "Journey_Purpose_of_Driver",
    "Weather_Conditions",
    "Road_Type",
    "Day_of_Week",
    "Light_Conditions",
    "Urban_or_Rural_Area",
    "Speed_limit",
    "Engine_Capacity"
]
df = df.select(*selected_cols)


In [20]:
# 결측치 비율 확인

from pyspark.sql.functions import col, count, when

total_count = df.count()

missing_df = df.select([
    (count(when(col(c).isNull(), c)) / total_count).alias(c + '_missing') for c in df.columns
])

missing_df.show(truncate=False)


+-------------+--------------------------+---------------------+--------------------+---------------------------------+--------------------------+-----------------+-------------------+------------------------+---------------------------+-------------------+-----------------------+
|label_missing|Age_Band_of_Driver_missing|Sex_of_Driver_missing|Vehicle_Type_missing|Journey_Purpose_of_Driver_missing|Weather_Conditions_missing|Road_Type_missing|Day_of_Week_missing|Light_Conditions_missing|Urban_or_Rural_Area_missing|Speed_limit_missing|Engine_Capacity_missing|
+-------------+--------------------------+---------------------+--------------------+---------------------------------+--------------------------+-----------------+-------------------+------------------------+---------------------------+-------------------+-----------------------+
|0.0          |0.0                       |0.0                  |0.0                 |0.0                              |0.0                       |0.0     

In [23]:
# 이상값 점검 컬럼 목록

categorical_cols = [
    "Age_Band_of_Driver",
    "Sex_of_Driver",
    "Vehicle_Type",
    "Journey_Purpose_of_Driver",
    "Weather_Conditions",
    "Road_Type",
    "Day_of_Week",
    "Light_Conditions",
    "Urban_or_Rural_Area",
    "Speed_limit"
]

from pyspark.sql.functions import col

for c in categorical_cols:
    print(f"📊 {c} 값 분포")
    df.groupBy(col(c)).count().orderBy("count", ascending=False).show(truncate=False)


📊 Age_Band_of_Driver 값 분포
+----------------------------+------+
|Age_Band_of_Driver          |count |
+----------------------------+------+
|26 - 35                     |424291|
|36 - 45                     |408773|
|46 - 55                     |331194|
|21 - 25                     |226532|
|56 - 65                     |195291|
|16 - 20                     |165772|
|Data missing or out of range|162771|
|66 - 75                     |87222 |
|Over 75                     |51996 |
|11 - 15                     |3559  |
|6 - 10                      |882   |
|0 - 5                       |125   |
+----------------------------+------+

📊 Sex_of_Driver 값 분포
+----------------------------+-------+
|Sex_of_Driver               |count  |
+----------------------------+-------+
|Male                        |1383012|
|Female                      |602458 |
|Not known                   |72870  |
|Data missing or out of range|68     |
+----------------------------+-------+

📊 Vehicle_Type 값 분포
+----------

In [24]:
from pyspark.sql.functions import when, col

df = df.withColumn(
    "Age_Band_of_Driver",
    when(col("Age_Band_of_Driver") == "Data missing or out of range", "Unknown_Age")
    .otherwise(col("Age_Band_of_Driver"))
).withColumn(
    "Sex_of_Driver",
    when(col("Sex_of_Driver").isin("Not known", "Data missing or out of range"), "Unknown_Sex")
    .otherwise(col("Sex_of_Driver"))
).withColumn(
    "Vehicle_Type",
    when(col("Vehicle_Type").rlike("(?i)(unknown|other|Data missing)"), "Other_Vehicle")
    .otherwise(col("Vehicle_Type"))
).withColumn(
    "Journey_Purpose_of_Driver",
    when(col("Journey_Purpose_of_Driver").isin("Not known", "Other", "Other/Not known (2005-10)", "Data missing or out of range"), "Unknown_Purpose")
    .otherwise(col("Journey_Purpose_of_Driver"))
).withColumn(
    "Road_Type",
    when(col("Road_Type").isin("Unknown", "Data missing or out of range"), "Unknown_Road")
    .otherwise(col("Road_Type"))
).withColumn(
    "Light_Conditions",
    when(col("Light_Conditions").isin("Darkness - lighting unknown", "Data missing or out of range"), "Unknown_Light")
    .otherwise(col("Light_Conditions"))
).withColumn(
    "Urban_or_Rural_Area",
    when(col("Urban_or_Rural_Area") == "Unallocated", "Unknown_Area")
    .otherwise(col("Urban_or_Rural_Area"))
).withColumn(
    "Speed_limit",
    when(col("Speed_limit").cast("string").isin("NA", "0", "10", "15"), "Unknown_Speed")
    .otherwise(col("Speed_limit"))
)


In [29]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

categorical_cols = [
    "Age_Band_of_Driver",
    "Sex_of_Driver",
    "Vehicle_Type",
    "Journey_Purpose_of_Driver",
    "Weather_Conditions",
    "Road_Type",
    "Day_of_Week",
    "Light_Conditions",
    "Urban_or_Rural_Area",
    "Speed_limit"
]

# 인덱서 + 인코더 정의
indexers = [StringIndexer(inputCol=c, outputCol=c + "_idx", handleInvalid="keep") for c in categorical_cols]
encoders = [OneHotEncoder(inputCol=c + "_idx", outputCol=c + "_vec") for c in categorical_cols]

# 파이프라인 구성
pipeline = Pipeline(stages=indexers + encoders)



In [30]:
from pyspark.sql.functions import col

df_encoded = df_encoded.withColumn("Engine_Capacity", col("Engine_Capacity").cast("double"))

In [35]:
# Feature Vector

from pyspark.ml.feature import VectorAssembler

# 벡터로 묶을 feature 목록 (OneHot 된 것들 + Engine_Capacity)
feature_cols = [c + "_vec" for c in categorical_cols] + ["Engine_Capacity"]



In [36]:
from pyspark.sql.functions import mean

# 평균값 계산
mean_val = df_encoded.select(mean("Engine_Capacity")).first()[0]

# null → 평균값으로 대체
df_encoded = df_encoded.fillna({"Engine_Capacity": mean_val})

In [37]:
#  feature 벡터와 label
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
final_df = assembler.transform(df_encoded)
# 확인
final_df.select("label", "features").show(5, truncate=False)

+-----+---------------------------------------------------------------------------------------------------+
|label|features                                                                                           |
+-----+---------------------------------------------------------------------------------------------------+
|0.0  |(75,[1,12,18,34,38,47,54,60,64,67,74],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,8268.0])            |
|0.0  |(75,[0,12,19,33,38,46,53,60,64,67,74],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2028.2906977580221])|
|0.0  |(75,[1,12,15,33,38,46,53,60,64,67,74],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4266.0])            |
|0.0  |(75,[7,12,15,33,38,47,52,59,64,67,74],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5343.0])            |
|0.0  |(75,[5,13,19,33,38,46,55,60,64,67,74],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,124.0])             |
+-----+---------------------------------------------------------------------------------------------------+
only showing top 5 rows



In [38]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10)
lr_model = lr.fit(final_df)

# 예측
predictions = lr_model.transform(final_df)
predictions.select("label", "prediction", "probability").show(5, truncate=False)


+-----+----------+-------------------------------------------------------------+
|label|prediction|probability                                                  |
+-----+----------+-------------------------------------------------------------+
|0.0  |0.0       |[0.8807610061583768,0.10524041366267523,0.013998580178948044]|
|0.0  |0.0       |[0.7458979141346158,0.24265222733673517,0.011449858528649048]|
|0.0  |0.0       |[0.8591533513969193,0.12818187346429988,0.012664775138780763]|
|0.0  |0.0       |[0.889272210654165,0.10242995593093472,0.008297833414900311] |
|0.0  |0.0       |[0.7823342496584101,0.21143943898316905,0.006226311358420819]|
+-----+----------+-------------------------------------------------------------+
only showing top 5 rows



In [39]:
# 정확도, 정밀도, 재현율, F1-score

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})
f1 = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})

print(f"정확도: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"정밀도: {precision:.4f}")
print(f"재현율: {recall:.4f}")


정확도: 0.8580
F1 Score: 0.7944
정밀도: 0.8018
재현율: 0.8580


In [40]:
# 혼동행렬
predictions.groupBy("label", "prediction").count().orderBy("label", "prediction").show()

+-----+----------+-------+
|label|prediction|  count|
+-----+----------+-------+
|  0.0|       0.0|1763762|
|  0.0|       1.0|   1875|
|  0.0|       2.0|     13|
|  1.0|       0.0| 263121|
|  1.0|       1.0|   2298|
|  1.0|       2.0|      7|
|  2.0|       0.0|  26867|
|  2.0|       1.0|    463|
|  2.0|       2.0|      2|
+-----+----------+-------+



## 해석
- Slight(0.0): 85% 이상   
- Serious(1.0) + Fatal(2.0): 15% 이하   
- 1.0 / 2.0 분류가 거의 안 됨

## 결론
- 모델이 거의 다 Slight (0.0)로만 예측함   
- 불균형 문제가 심각함   
- Serious와 Fatal은 거의 전혀 분류되지 않음   

→ 이 상태로는 정책/의사결정용 분석에 위험

## 다음 액션
- Slight vs Serious+Fatal 이진 분류
- `label_binary = 0 if label == 0.0 else 1`   
- 0: Slight (안전) / 1: Serious or Fatal (심각 사고)   


In [41]:
spark.stop()